In [18]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../../../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import os
from src.utils.train_utils import set_cuda_env, EcgDataModule, get_dummy_hparams, get_common_trainer_params, flatten_dict
from src.basic.constants import TRAIN_LOG_PATH
from src.models.ecg_step_module import EcgPipeline
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

BATCH_SIZE = 32
MAX_EPOCHS = 20

SAVE_DIR = os.path.join(TRAIN_LOG_PATH, "hard_rule/")
datamodule = EcgDataModule(batch_size=256, num_workers=0)
datamodule.setup('fit')
datamodule.hparams.batch_size = BATCH_SIZE
print('Using batch size: ', datamodule.hparams.batch_size)
train_dl, val_dl = datamodule.train_dataloader(), datamodule.val_dataloader()

train dataset loaded!
val dataset loaded!
Using batch size:  32


In [21]:
set_cuda_env(gpu_ids='6')

hparams = get_dummy_hparams()
hparams['is_using_hard_rule'] = True
model = EcgPipeline(hparams)
print("is_using_hard_rule: ", model.is_using_hard_rule)

checkpoint_callback = ModelCheckpoint(save_top_k=3,
                                      monitor="val_metrics/auroc",
                                      mode="max",
                                      save_last=True,
                                      filename="epoch={epoch}-step={step}-auroc={val_metrics/auroc:.7f}",
                                      auto_insert_metric_name=False)

checkpoint_callback.CHECKPOINT_NAME_LAST = "epoch={epoch}-step={step}-last"
trainer = Trainer(
    callbacks=[checkpoint_callback],
    logger=TensorBoardLogger(save_dir=SAVE_DIR),
    max_epochs=MAX_EPOCHS,
    # limit_train_batches=2,
    # limit_val_batches=2,
    **get_common_trainer_params())

# record hyperparameters
trainer.logger.log_hyperparams(flatten_dict(hparams))

# trainer.tune(model, datamodule=datamodule)
# if datamodule.hparams.batch_size <= 16:
#     raise torch.cuda.OutOfMemoryError("Batch size <= 16, it's likely that OOM Error has occur")
# if len(datamodule.train_ds) % datamodule.hparams.batch_size == 1:
#     datamodule.hparams.batch_size -= 1


trainer.fit(model, train_dl, val_dl)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name          | Type             | Params
----------------------------------------------------
0  | loss_fn       | BCELoss          | 0     
1  | pipeline      | SeqSteps         | 5.3 K 
2  | NORM_ensemble | Linear           | 10    
3  | LBBB_ensemble | Linear           | 3     
4  | IMI_ensemble  | Linear           | 5     
5  | AMI_ensemble  | Linear           | 6     
6  | LMI_ensemble  | Linear           | 5     
7  | LVH_ensemble  | Linear           | 6     
8  | RVH_ensemble  | Linear           | 5     
9  | train_metric  | MetricCollection | 0     
10 | val_metric    | MetricCollection | 0     
11 | test_metric   | MetricCollection | 0     
----------------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021  

is_using_hard_rule:  True


/Users/tf/miniconda3/envs/ecg_xai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/tf/miniconda3/envs/ecg_xai/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in macro-average
  warnings.warn(*args, **kwargs)
/Users/tf/miniconda3/envs/ecg_xai/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/Users/tf/miniconda3/envs/ecg_xai/lib/python3.10/site-packages/pytorch_light

In [23]:
trainer.max_epochs = 40

AttributeError: can't set attribute 'max_epochs'